In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

# -----------------------------
# Paths
# -----------------------------
base_dir = "/kaggle/input/original_dataset"  # Original dataset with 3 class folders
output_dir = "/kaggle/working/augmented_dataset"

classes = ['polyps', 'esophagitis', 'ulcerative-colitis']
IMG_SIZE = (224, 224)  # Resize images to 224x224

# -----------------------------
# ImageDataGenerator for augmentation
# -----------------------------
datagen = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# -----------------------------
# Create output folders
# -----------------------------
for cls in classes:
    os.makedirs(os.path.join(output_dir, cls), exist_ok=True)

# -----------------------------
# Augmentation loop
# -----------------------------
for cls in classes:
    class_input_path = os.path.join(base_dir, cls)
    class_output_path = os.path.join(output_dir, cls)
    
    image_files = [f for f in os.listdir(class_input_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    for img_file in tqdm(image_files, desc=f"Processing {cls}"):
        img_path = os.path.join(class_input_path, img_file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, IMG_SIZE)
        img = np.expand_dims(img, 0)
        
        # Generate augmented images until we reach ~4500 per class
        count = 0
        for batch in datagen.flow(img, batch_size=1, save_to_dir=class_output_path,
                                  save_prefix='aug', save_format='jpg'):
            count += 1
            if count >= 4:  # 1 original + 4 augmented = 5 images per original
                break
